In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy
import scipy as sp
from scipy import sparse, spatial

%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

In [ ]:
domain = [(-1, 1), (-1, 1)]
num_points = [10, 10]

discretization = np.meshgrid(np.linspace(domain[0][0], domain[0][1], num_points[0]),
                             np.linspace(domain[1][0], domain[1][1], num_points[1]))

discretization = np.array([x.ravel() for x in discretization]).T

#### Get matrix for values of test points

For each vertex x, the next state is given by some function $f(x)$. The goal is to find the value $V(f(x))$ as a function of the values at the other vertices. This value can then be used as a constraint in the optimization problem.
$$V(f(x)) = a^\mathrm{T} V(\mathrm{vertices})$$

We start by finding the simplex corresponding to $f(x)$. Given this simplex's vertices, we solve for $a$ above, using one arbitrary vertex as a reference point:
$$V(f(x)) = V(v_1) + a_1 V(v_2) + a_2 V(v_3)$$

The hyperplane spanned by the simplex is given by
\begin{align*}
\xi_1 ( x_2 - x_1) + \xi_2 (y_2 - y_1) = V(v_2) - V(v_1) \\
\xi_1 ( x_3 - x_1) + \xi_2 (y_3 - y_1) = V(v_3) - V(v_1) 
\end{align*}

and, as a result,
$$V(f(x_t)) = V(v_1) + (x_t - x_1, y_t - y_1)
\left( \begin{matrix}
x_2 - x_1 & y_2 - y_1 \\
x_3 - x_1 & y_3 - y_1
\end{matrix} \right)^{-1}
\left( \begin{matrix}
V(v_2) - V(v_1) \\
V(v_3) - V(v_1)
\end{matrix} \right),
$$
which can easily be solved for $a_1$ and $a_2$.

In [ ]:
class Triangulization(sp.spatial.Delaunay):
    """
    Generalization of Delaunay triangulization with additional properties.
    
    A normal Delaunay triangulation, but provides additional methods to obtain
    the hyperplanes and gradients.
    
    Parameters
    ----------
    see scipy.spatial.Delaunay
    """
    
    def __init__(self, points):
        super(Triangulization, self).__init__(points)

    def function_values_at(self, points):
        """
        Obtain function values at points from triangulation.

        Get a matrix that, when multiplied with the vector of function values
        on the vertices of the simplex, returns the function values at points.

        Parameters
        ----------
        points: 2d array
            Each row represents one points

        Returns
        -------
        B: scipy.sparse
            A sparse matrix so that V(points) = B * V(vertices)
        """
        simplex_ids = self.find_simplex(points)
        simplices = self.simplices[simplex_ids]

        num_constraints = len(points) * 3
        X = np.empty(num_constraints, dtype=np.float)
        I = np.empty(num_constraints, dtype=np.int32)
        J = np.empty(num_constraints, dtype=np.int32)

        for i, (point, simplex) in enumerate(zip(points, simplices)):
            simplex_points = points[simplex]
            # TODO: Add check for when it is outside the triangulization

            tmp = np.linalg.solve((simplex_points[1:] - simplex_points[:1]).T, point - simplex_points[0])

            index = slice(3 * i, 3 * (i + 1))
            X[index] = [1 - np.sum(tmp), tmp[0], tmp[1]]
            I[index] = i
            J[index] = simplex

        return sparse.coo_matrix((X, (I, J)), shape=(len(points), self.npoints)).tocsr()

In [ ]:
triangulation = Triangulization(discretization)

plt.triplot(triangulation.points[:, 0], triangulation.points[:, 1], triangulation.simplices.copy())
plt.plot(triangulation.points[:, 0], triangulation.points[:, 1], 'o')
plt.ylim(domain[0])
plt.xlim(domain[1])
plt.show()

In [ ]:

B = triangulation.function_values_at(discretization)
print(B.todense().dot(np.random.randn(len(discretization), 1)))

# The following ist just a random example of how cvxpy works

In [ ]:
# Generate data for long only portfolio optimization.
np.random.seed(1)
n = 10
mu = np.abs(np.random.randn(n, 1))
Sigma = np.random.randn(n, n)
Sigma = Sigma.T.dot(Sigma)

# Long only portfolio optimization.
# from cvxpy import *
w = cvxpy.Variable(n)
gamma = cvxpy.Parameter(sign='positive')
ret = mu.T*w 
risk = cvxpy.quad_form(w, Sigma)
prob = cvxpy.Problem(cvxpy.Maximize(ret - gamma*risk), 
                    [cvxpy.sum_entries(w) == 1, 
                     w >= 0])

# Compute trade-off curve.
SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)
for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    risk_data[i] = np.sqrt(risk.value)
    ret_data[i] = ret.value

# Plot long only trade-off curve.

markers_on = [29, 40]
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(risk_data, ret_data, 'g-')
for marker in markers_on:
    plt.plot(risk_data[marker], ret_data[marker], 'bs')
    ax.annotate(r"$\gamma = %.2f$" % gamma_vals[marker], xy=(risk_data[marker]+.08, ret_data[marker]-.03))
for i in range(n):
    plt.plot(np.sqrt(Sigma[i,i]), mu[i], 'ro')
plt.xlabel('Standard deviation')
plt.ylabel('Return')
plt.show()